In [1]:
import os
import os.path
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

2022-09-07 02:15:20.027546: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-07 02:15:20.027564: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
x = []
y = []

for dirpath, dirnames, filenames in os.walk("../Fall Data (New)/"):
    for filename in [f for f in filenames if f.endswith(".txt")]:
        df = pd.read_csv(os.path.join(dirpath, filename),header=None)
        fall_time = df[df[1].isna()]
        df = df[df[1].notna()]
        ax = np.array(df[1])
        ay = np.array(df[2])
        az = np.array(df[3])
        gx = np.array(df[4])
        gy = np.array(df[5])
        gz = np.array(df[6])
        time = np.array(df[0])
        a = np.sqrt(ax**2 + ay**2 + az**2)
        if filename[0] == "A":
            # ADL file
            for i in range(a.shape[0]):
                if a[i] >= 1.6:
                    st = i - 450
                    ed =  i + 450
                    if st < 0 or ed >= a.shape[0]:
                        continue

                    tmp = np.array(list(zip(ax[st:ed],ay[st:ed],az[st:ed],gx[st:ed],gy[st:ed],gz[st:ed])))


                    x.append(tmp)
                    y.append(1)
        elif filename[11] == "B":
            continue

        else:
            # Fall Data
            i = np.argmax(a)
#             val = int(np.array(fall_time[0])[1])
#             i = np.where(time == val)[0][0]
            st = i - 450
            ed =  i + 450
            if st < 0 or ed >= a.shape[0] or a[i] < 1.6:
#                 print(filename)
                print(filename,st,ed,a[i])
                continue
            print(filename)
            tmp = np.array(list(zip(ax[st:ed],ay[st:ed],az[st:ed],gx[st:ed],gy[st:ed],gz[st:ed])))
            x.append(tmp)
            y.append(0)


TESTING_ML_v58.txt
TESTING_ML_v55.txt
TESTING_ML_v49.txt
TESTING_ML_v9.txt
TESTING_ML_v19.txt
TESTING_ML_v29.txt
TESTING_ML_v57.txt
TESTING_ML_v25.txt
TESTING_ML_v50.txt
TESTING_ML_v23.txt
TESTING_ML_v30.txt
TESTING_ML_v56.txt
TESTING_ML_v53.txt
TESTING_ML_v28.txt
TESTING_ML_v40.txt
TESTING_ML_v16.txt
TESTING_ML_v20.txt -78 822 4.7132685049761385
TESTING_ML_v21.txt
TESTING_ML_v59.txt
TESTING_ML_v8.txt
TESTING_ML_v52.txt
TESTING_ML_v15.txt
TESTING_ML_v22.txt
TESTING_ML_v51.txt
TESTING_ML_v54.txt
TESTING_ML_v10.txt -102 798 12.53990430585497


In [5]:
from collections import Counter
cnt =  Counter(y)
cnt

Counter({0: 24, 1: 9446})

In [6]:
X = np.array(x)
Y = np.array(y)
X = X.reshape(-1,900,6,1)
# X_flat = X.reshape(-1,5400)
print(X.shape,Y.shape)


(9470, 900, 6, 1) (9470,)


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=50)
X_flat_train = X_train.reshape(-1,5400)
X_flat_test = X_test.reshape(-1,5400)
y_train_cat = keras.utils.to_categorical(y_train)
y_test_cat = keras.utils.to_categorical(y_test)
print(X_test.shape)


(1894, 900, 6, 1)


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
model = Sequential()
model.add(Conv2D(10, kernel_size=(50,6), activation='relu', input_shape=(900,6,1),strides=50))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
adam = tf.keras.optimizers.Adam(learning_rate=0.001,)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 18, 1, 10)         3010      
                                                                 
 flatten (Flatten)           (None, 180)               0         
                                                                 
 dense (Dense)               (None, 2)                 362       
                                                                 
Total params: 3,372
Trainable params: 3,372
Non-trainable params: 0
_________________________________________________________________


2022-09-07 02:19:27.391017: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-07 02:19:27.391379: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-07 02:19:27.391431: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-09-07 02:19:27.391480: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-09-07 02:19:27.391528: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [9]:
model.fit(X_train, y_train_cat, validation_split=0.25, epochs=20,batch_size=128)
results = model.evaluate(X_test, y_test_cat, batch_size=128)

Epoch 1/20
45/45 [==============================] - 1s 7ms/step - loss: 0.0929 - accuracy: 0.9681 - val_loss: 0.0372 - val_accuracy: 0.9963
Epoch 2/20
45/45 [==============================] - 0s 4ms/step - loss: 0.0169 - accuracy: 0.9982 - val_loss: 0.0321 - val_accuracy: 0.9963
Epoch 3/20
45/45 [==============================] - 0s 4ms/step - loss: 0.0100 - accuracy: 0.9982 - val_loss: 0.0277 - val_accuracy: 0.9963
Epoch 4/20
45/45 [==============================] - 0s 4ms/step - loss: 0.0056 - accuracy: 0.9988 - val_loss: 0.0246 - val_accuracy: 0.9963
Epoch 5/20
45/45 [==============================] - 0s 4ms/step - loss: 0.0036 - accuracy: 0.9995 - val_loss: 0.0225 - val_accuracy: 0.9963
Epoch 6/20
45/45 [==============================] - 0s 4ms/step - loss: 0.0022 - accuracy: 0.9996 - val_loss: 0.0200 - val_accuracy: 0.9963
Epoch 7/20
45/45 [==============================] - 0s 4ms/step - loss: 0.0015 - accuracy: 0.9998 - val_loss: 0.0185 - val_accuracy: 0.9968
Epoch 8/20
45/45 [==

In [10]:
print(y_test.shape)

(1894,)


In [11]:
from sklearn.metrics import confusion_matrix


predictions = model.predict(X_test, batch_size=128, verbose=1)
classes_x=(np.argmax(predictions,axis=1))
print(predictions)
true_classes = y_test

# y_test_flat = np.argmax(y_test, axis=-1)
c = confusion_matrix(y_test, classes_x)
print('Confusion matrix:\n', c)
print('sensitivity', c[0, 0] / (c[0, 1] + c[0, 0]))
print('specificity', c[1, 1] / (c[1, 1] + c[1, 0]))

15/15 [==============================] - 0s 1ms/step
[[2.8436039e-08 9.9999994e-01]
 [1.5591735e-06 9.9999839e-01]
 [9.1682200e-09 9.9999994e-01]
 ...
 [2.5170602e-08 1.0000000e+00]
 [1.2425081e-05 9.9998760e-01]
 [3.1964351e-09 1.0000000e+00]]
Confusion matrix:
 [[   2    3]
 [   0 1889]]
sensitivity 0.4
specificity 1.0


In [12]:
ind = 0
for pa,pb in zip(classes_x,y_test):
    if pa != pb:
        print(ind,pa,pb)
    ind += 1

101 1 0
687 1 0
1217 1 0


In [13]:
list(X_test[497][450])

[array([3.55]),
 array([3.86]),
 array([1.72]),
 array([4.73]),
 array([3.92]),
 array([-4.63])]

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
model = Sequential()
# model.add(Conv2D(10, kernel_size=(5,6), activation='relu', input_shape=(900,6,1),strides=3))
model.add(Dense(100, activation='relu',input_dim=5400))
model.add(Dense(10, activation='relu'))

# model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
adam = tf.keras.optimizers.Adam(learning_rate=0.001,)
model.compile(optimizer=adam, loss='BinaryCrossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 100)               540100    
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 541,121
Trainable params: 541,121
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(X_flat_train, y_train, validation_split=0.25, epochs=20,batch_size=128)
results = model.evaluate(X_flat_test, y_test, batch_size=128)

Epoch 1/20
45/45 [==============================] - 1s 7ms/step - loss: 0.0955 - accuracy: 0.9696 - val_loss: 0.0513 - val_accuracy: 0.9968
Epoch 2/20
45/45 [==============================] - 0s 5ms/step - loss: 0.0019 - accuracy: 0.9996 - val_loss: 0.0536 - val_accuracy: 0.9963
Epoch 3/20
45/45 [==============================] - 0s 4ms/step - loss: 4.0766e-04 - accuracy: 0.9996 - val_loss: 0.0514 - val_accuracy: 0.9963
Epoch 4/20
45/45 [==============================] - 0s 5ms/step - loss: 3.0457e-04 - accuracy: 0.9996 - val_loss: 0.0513 - val_accuracy: 0.9963
Epoch 5/20
45/45 [==============================] - 0s 4ms/step - loss: 2.8428e-04 - accuracy: 0.9996 - val_loss: 0.0513 - val_accuracy: 0.9963
Epoch 6/20
45/45 [==============================] - 0s 5ms/step - loss: 2.7642e-04 - accuracy: 0.9996 - val_loss: 0.0512 - val_accuracy: 0.9963
Epoch 7/20
45/45 [==============================] - 0s 4ms/step - loss: 2.8249e-04 - accuracy: 0.9996 - val_loss: 0.0515 - val_accuracy: 0.9963


In [16]:
orig_shape = X.shape

wX = X.reshape(-1,5400)
print(X.shape,Y.shape,wX.shape)

(9470, 900, 6, 1) (9470,) (9470, 5400)


In [17]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
# transform the dataset
new_X, new_Y = pipeline.fit_resample(wX, Y)

In [18]:
new_X = new_X.reshape((-1, 900, 6, 1))
print(new_X.shape,new_Y.shape)

(2832, 900, 6, 1) (2832,)


In [19]:
from collections import Counter
cnt =  Counter(new_Y)
cnt

Counter({0: 944, 1: 1888})

In [20]:
from sklearn.model_selection import train_test_split
new_X_train, new_X_test, new_y_train, new_y_test = train_test_split(new_X, new_Y, test_size=0.2, random_state=32)
new_y_train_cat = keras.utils.to_categorical(new_y_train)
new_y_test_cat = keras.utils.to_categorical(new_y_test)

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
model = Sequential()
# 100%
# model.add(Conv2D(20, kernel_size=(50,6), activation='relu', input_shape=(900,6,1),strides=5)) 
model.add(Conv2D(7, kernel_size=(100,6), activation='relu', input_shape=(900,6,1),strides=5))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
adam = tf.keras.optimizers.Adam(learning_rate=0.001,)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 161, 1, 7)         4207      
                                                                 
 flatten_3 (Flatten)         (None, 1127)              0         
                                                                 
 dense_6 (Dense)             (None, 2)                 2256      
                                                                 
Total params: 6,463
Trainable params: 6,463
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.fit(new_X_train, new_y_train_cat, validation_split=0.25, epochs=20,batch_size=128)
results = model.evaluate(new_X_test, new_y_test_cat, batch_size=128)

Epoch 1/20
14/14 [==============================] - 1s 30ms/step - loss: 0.3047 - accuracy: 0.8610 - val_loss: 0.0657 - val_accuracy: 0.9859
Epoch 2/20
14/14 [==============================] - 0s 24ms/step - loss: 0.0247 - accuracy: 0.9959 - val_loss: 0.0080 - val_accuracy: 1.0000
Epoch 3/20
14/14 [==============================] - 0s 25ms/step - loss: 0.0069 - accuracy: 1.0000 - val_loss: 0.0032 - val_accuracy: 1.0000
Epoch 4/20
14/14 [==============================] - 0s 25ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 5/20
14/14 [==============================] - 0s 25ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 6/20
14/14 [==============================] - 0s 25ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 7/20
14/14 [==============================] - 0s 25ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 8/20
14

In [29]:
from sklearn.metrics import confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=3372)
#print(X_test.shape)
predictions = model.predict(X, batch_size=128, verbose=0)
classes_x=(np.argmax(predictions,axis=1))
#print(predictions)
true_classes = Y

# y_test_flat = np.argmax(y_test, axis=-1)
c = confusion_matrix(Y, classes_x)
print('Confusion matrix:\n', c)
# print('Accuracy', )
print('Accuracy:{}%'.format((c[0, 0] + c[1, 1]) * 100 / (c[0, 1] + c[0, 0] + c[1, 1] + c[1, 0])))
print('Sensitivity:{}%'.format(c[0, 0]*100 / (c[0, 1] + c[0, 0])))
print('Specificity:{}%'.format(c[1, 1]*100 / (c[1, 1] + c[1, 0])))
# print('sensitivity', c[0, 0] / (c[0, 1] + c[0, 0]))
# print('specificity', c[1, 1] / (c[1, 1] + c[1, 0]))

Confusion matrix:
 [[  24    0]
 [   0 9446]]
Accuracy:100.0%
Sensitivity:100.0%
Specificity:100.0%
